In [ ]:
import requests
import json
import pandas as pd
import os
import csv

In [ ]:
stock_path = ['Stock_Price_List.csv']
for a in stock_path:
    if os.path.exists(a):
        os.remove(a)

In [ ]:
def get_web(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.27 Safari/537.36 Edg/99.0.1150.16'}
    response = requests.get(url, headers=headers)
    return response

In [ ]:
def get_stock_pice(stock_code, interval_day):
    url = 'https://finfo-api.vndirect.com.vn/v4/stock_prices?sort=date&q=code:' + stock_code
    data = get_web(url).json()
    page_size = data['totalElements']
    
    #Resize array
    url = url + '&size=' + str(page_size)
    data = get_web(url).json()

    cate_list = []
    i = 0
    for d in data['data']:
        if i < interval_day:
            try:
                date_time = d['date']
                stock_type = d['type']
                stock_floor = d['floor']
                stock_id = d['code']
                open_price = d['adOpen']
                close_price = d['adClose']
                min_price = d['adLow']
                max_price = d['adHigh']
                avg_price = d['adAverage']
                vol = d['nmVolume']
                value = d['nmValue']

                cate_list.append([date_time, stock_type, stock_floor, stock_id, open_price, close_price, min_price, max_price, avg_price, vol, value])
            except Exception as err:
                print(err)
            i += 1
    
    return cate_list

In [ ]:
%%time 

df = pd.read_csv('Stock_List.csv')
stock_list = df.loc[(df['stock_type']=='STOCK') & (df['stock_status']=='listed')]['stock_code'].to_list()
interval_day = 5

header = [['date_time', 'stock_type', 'stock_floor', 'stock_code', 'open_price', 'close_price', 'min_price', 'max_price', 'avg_price', 'volume', 'value']]
detail = []
for s in stock_list:
    data = get_stock_pice(s, interval_day)
    for d in data:
        detail.append(d)
    
with open(stock_path[0], 'w', newline='', encoding='utf-8-sig') as f:
    write = csv.writer(f)
    write.writerows(header)
    write.writerows(detail)
    
print('Crawl done!')